In [3]:
import random
import time
import uuid



In [4]:
def randomInt():
    return random.randint(-2147483648, 2147483647)

In [5]:
deviceNameLength = 32
serviceNameLength = 8

devicePort = 8080

In [6]:
def generateName(length=5):
    return str(uuid.uuid4().hex)[:length]



In [15]:
class ArduinoDevice:
    def __init__(self, name, controllers, rx=8, tx=9):
        self.rx = rx
        self.tx = tx
        self.name = name
        self.serial = randomInt()
        self.type = "arduino"
        self.port = devicePort
        self.controllers = controllers
        
        

In [16]:
intypes = ["analog", "temperature11", "humidity11", "temperature22", "humidity22", "alert", "pressure"]

In [17]:
ids = {'analog': 1000,
 'onoff': 1001,
 'temperature11': 1002,
 'humidity11': 1003,
 'alert': 1004,
 'temperature22': 1006,
 'humidity22': 1007,
 'pressure': 1008}

serviceIds = {
    'analog': "ANALOG_ID",
 'onoff':"ONOFF_ID",
 'temperature11': "TEMPERATURE_DHT11_ID",
 'humidity11': "HUMIDITY_DHT11_ID",
 'alert': "DIGITAL_ALERT_ID",
 'temperature22': "TEMPERATURE_DHT22_ID",
 'humidity22': "HUMIDITY_DHT22_ID",
 'pressure': "PRESSURE_ID"
}


In [18]:
class ArduinoController:
    def __init__(self, _type, pin):
        self._type =ids[_type]
        self.rawType = _type
        self.serviceId = serviceIds[_type]
        self.defineService = serviceIds[_type][:-3]
        self.pin = pin
        self.serial = randomInt()
        
        

In [30]:
def mode(controller):
    return "INPUT" if controller.rawType in intypes else "OUTPUT"

In [31]:
device = ArduinoDevice(name="1", controllers=[
#     ArduinoController("analog", "A3"),
#     ArduinoController("alert", "10"),
#     ArduinoController("alert", "12"),
#     ArduinoController("temperature22", "7"),
#     ArduinoController("humidity22", "7"),
    ArduinoController("onoff", "4"),
    ArduinoController("onoff", "3"),
    ArduinoController("onoff", "2")
])

In [32]:
paste = """
#include "user_properties.h"
#include "services.h"
#include "service.h"
#include "constants.h"
#include "jsonUtil.h"
#include "util.h"
#include "streamLengthCounter.h"

#ifdef __ets__
#include "ets_sys.h"
#include "osapi.h"
#endif

const IPAddress RASPBERRY_IP = IPAddress(192,168,0,102);
const int RASPBERRY_PORT  = 8080;
const int ARDUINO_PORT = 80;

"""
paste += "const int DEVICE_SERIAL = " + str(device.serial) + ";\n\n"

for i in set(map(lambda x: x.defineService, device.controllers)):
    paste += "#define " + i + "\n"
    
paste += "\n// pins for communication with esp8266\n"
paste += "#define RX " + str(device.rx) + " // attach to TX of esp\n"
paste += "#define TX " + str(device.tx) + " // attach to RX of esp\n"

paste += "Service services[" + str(len(device.controllers)) + "];\n"
paste += "void setupConfiguration() {\n"

for controller in device.controllers:
    paste += "\tpinMode(" + str(controller.pin) + ", " + mode(controller) + ");\n"
    paste += "\tservices[%i].pin = %s;\n" % (device.controllers.index(controller), controller.pin)
    paste += "\tservices[%i].type = %s;\n" % (device.controllers.index(controller), controller._type)
    paste += "\tservices[%i].serial = %s;\n" % (device.controllers.index(controller), controller.serial)
    paste += "\n"
paste += "}\n"

print(paste)



#include "user_properties.h"
#include "services.h"
#include "service.h"
#include "constants.h"
#include "jsonUtil.h"
#include "util.h"
#include "streamLengthCounter.h"

#ifdef __ets__
#include "ets_sys.h"
#include "osapi.h"
#endif

const IPAddress RASPBERRY_IP = IPAddress(192,168,0,102);
const int RASPBERRY_PORT  = 8080;
const int ARDUINO_PORT = 80;

const int DEVICE_SERIAL = 460902108;

#define ONOFF

// pins for communication with esp8266
#define RX 8 // attach to TX of esp
#define TX 9 // attach to RX of esp
Service services[3];
void setupConfiguration() {
	pinMode(4, OUTPUT);
	services[0].pin = 4;
	services[0].type = 1001;
	services[0].serial = 907747194;

	pinMode(3, OUTPUT);
	services[1].pin = 3;
	services[1].type = 1001;
	services[1].serial = -9200262;

	pinMode(2, OUTPUT);
	services[2].pin = 2;
	services[2].type = 1001;
	services[2].serial = -688587091;

}

